# Processamento dos dados do extrato do BTG para capturar informações sobre BTC (aluguel de ações)


Eu gerei o extrato no site do BTG e após isso, selecionei as colunas de dados e copiei para o Excel.
Removi a coluna de saldo, uma vez que não preciso dela.
Não encontrei uma maneira de exportar o extrato em Excel.

In [ ]:
import pandas as pd

In [ ]:
def processa_descricao(row):
    saida = ""
    if "TAXA EMPRÉSTIMO" in row['Descrição']:
        saida = "TAXA DE REMUNERAÇÃO"
    elif "Corretagem BTC" in row['Descrição']:
        saida = "COMISSÃO BTC"
    elif "REEMB.DIVIDENDOS/JUROS" in row['Descrição']:
        saida = "REEMB.DIVIDENDOS"
    elif "TAXA NEGOCIACAO EMPRESTIMO - BTC" in row['Descrição']:
        saida = "TAXA NEGOCIACAO EMPRESTIMO"
    elif "TAXA DE PERMANÊNCIA OP. ESTRUTURADAS" in row['Descrição']:
        saida = "TAXA DE PERMANÊNCIA OP. ESTRUTURADAS"       
    else:
        saida = row['Descrição']
    
    return saida
        

In [ ]:
# path do arquivo de extrato
# formato do arquivo
# Data, Descrição, Operação, Movimentação

ARQUIVO_EXTRATO = r'./extrato_btg.xlsx'

In [ ]:
dados = pd.read_excel(ARQUIVO_EXTRATO)
#dados.drop(columns=['Saldo'], inplace=True)

In [ ]:
dados_btc = dados[(dados['Descrição'].str.contains('BTC')) | 
                  (dados['Descrição'].str.contains('EMPRESTIMO')) |
                  (dados['Descrição'].str.contains('TAXA')) |
                  (dados['Descrição'].str.contains('REEMB.DIVIDENDOS/JUROS'))
                 ]

#remove as linhas com o fundo BTCI11
dados_btc = dados_btc[(~dados_btc['Descrição'].str.contains('BTCI11'))]


In [ ]:
dados_btc['FILTRO'] = dados_btc.apply(processa_descricao, axis=1)


In [ ]:
dados_btc

## Realiza a união do dataframe `dados_btc` e `pivot_custos_totais`

In [ ]:
dados_btc2 = dados_btc.copy()
dados_btc2

In [ ]:
#dados_btc2['COMISSAO_BTC_PAPEL'] = dados_btc2.apply(calcula_custos_comissao, axis=1)
#dados_btc2

In [ ]:
# Define the regex pattern for extraction
pattern = r'LIQ BOLSA \(Corretagem BTC\s+\d+\s+(\w+)\)|LIQ BOLSA \(TAXA EMPRÉSTIMO\s+(\w+)\s+\d+\)'

# Extract the substrings using the regex pattern and create a new column
dados_btc2['ATIVO'] = dados_btc2['Descrição'].str.extract(pattern).bfill(axis=1).iloc[:, 0]

#TAXA DE PERMANÊNCIA OP. ESTRUTURADAS
# Apesar de não se um custo de BTC, vou colocar em ATIVO a definir, pois é um custo que preciso inserir manualmente na Sencon
dados_btc2.loc[dados_btc2['Descrição'].str.contains("TAXA DE PERMANÊNCIA OP. ESTRUTURADAS"), 'ATIVO'] = 'A DEFINIR'



In [ ]:
dados_btc2[dados_btc2['Data'] == '2023-08-16']

In [ ]:
dados_btc2['COMISSAO_BTC_PAPEL'] = 0
dados_btc2['REMUNERACAO_BTC_PAPEL'] = 0
dados_btc2['TAXA_PERMANENCIA_PAPEL'] = 0
dados_btc2['REEMB_DIV_PAPEL'] = 0


In [ ]:
# seleciona o valor da Taxa de negociação e coloca na mesma linha que a TAXA DE REMUNERACAO
# em geral, não haverá reembolso num mesmo dia que houve 
for index, row in dados_btc2.iterrows():
    if row['FILTRO'] == 'TAXA DE REMUNERAÇÃO':
        valor = dados_btc2.loc[(dados_btc2['Data']==row['Data']) & 
                           (dados_btc2['ATIVO']==row['ATIVO']) &
                           (dados_btc2['FILTRO']=='COMISSÃO BTC')]
        #pega o valor da linha 0, coluna 4 (coluna 4 é a coluna do Valor de transação)
        #print(f"x{row['Movimentação']}x y{row['ATIVO']}y z{valor}z")
        #print(f"primeiro: {len(valor)}")
        if len(valor) > 0:
            valor_escalar = valor.iat[0,3]
            dados_btc2.loc[index,'COMISSAO_BTC_PAPEL'] = valor_escalar


In [ ]:
# adiciona coluna de TAXA DE REMUNERAÇÃO
dados_btc2.loc[dados_btc2['FILTRO'] == "TAXA DE REMUNERAÇÃO", 'REMUNERACAO_BTC_PAPEL'] = dados_btc2["Movimentação"] 

# adiciona coluna de TAXA DE PERMANENCIA
dados_btc2.loc[dados_btc2['FILTRO'] == "TAXA DE PERMANÊNCIA OP. ESTRUTURADAS", 'TAXA_PERMANENCIA_PAPEL'] = dados_btc2["Movimentação"] 

# adiciona coluna de REEMB DIVIDENDOS
dados_btc2.loc[dados_btc2['FILTRO'] == "REEMB.DIVIDENDOS",'REEMB_DIV_PAPEL'] = dados_btc2['Movimentação'] 


# reposiciona as colunas para melhor organização
dados_btc2 = dados_btc2.reindex(['Data', 'ATIVO', 'FILTRO', 'Operação','Descrição','Movimentação','REMUNERACAO_BTC_PAPEL',
                                 'COMISSAO_BTC_PAPEL','REEMB_DIV_PAPEL' ,
                                'TAXA_PERMANENCIA_PAPEL'],axis=1)

# preenche com zero onde estiver NA
# se nao fizer isso, o CUSTO_TOTAL ficará com NA
dados_btc2.fillna(0, inplace=True)

# custo total
dados_btc2['CUSTO_TOTAL'] = dados_btc2['REMUNERACAO_BTC_PAPEL'] + \
                            dados_btc2['COMISSAO_BTC_PAPEL'] +    \
                            dados_btc2['REEMB_DIV_PAPEL'] +       \
                            dados_btc2['TAXA_PERMANENCIA_PAPEL'] 


dados_btc2.loc[dados_btc2['Operação']=="Crédito", 'CUSTO_TOTAL'] = dados_btc2['REMUNERACAO_BTC_PAPEL'] - \
                                                                   dados_btc2['COMISSAO_BTC_PAPEL'] 


# remove linhas de TAXA NEGOCIACAO EMPRESTIMO
dados_btc2 = dados_btc2[~(dados_btc2['FILTRO']=='COMISSÃO BTC')]

# remove colunas desnecessárias
dados_btc2.drop(columns=['FILTRO','Descrição','Movimentação'], inplace=True)


In [ ]:
dados_btc2[dados_btc2['Data'] == '2023-08-16']

In [ ]:
dados_btc2[dados_btc2['Data'] == '2023-08-22']

In [ ]:
# exporta dados para o excel
dados_btc2.to_excel('custos_btc_btg.xlsx')